# Beaver Tutorial 2: Advanced Features (Data Owner)

**Features:** Output capture, live variables, reliable wait methods

Run this alongside `02-advanced-features-ds.ipynb` in a separate tab.

## Step 1: Setup

In [1]:
!uv pip install pandas matplotlib numpy -q

In [2]:
import beaver
from beaver import Twin

bv = beaver.ctx()
session = bv.active_session()

print(f"You: {bv.user}")
print(f"Peer: {session.peer}")

🔄 Auto-load replies enabled for client1@sandbox.local (polling every 2.0s)
🟢 Active session loaded: 7f01f8db8e4f
   Peer: client2@sandbox.local
You: client1@sandbox.local
Peer: client2@sandbox.local


## Step 2: Create Stock Data

In [3]:
import pandas as pd
import numpy as np

dates = pd.date_range('2024-01-01', periods=30, freq='D')

# Mock data (safe to share)
np.random.seed(42)
mock_stocks = pd.DataFrame({
    'AAPL': 150 + np.random.randn(30).cumsum() * 2,
    'GOOGL': 140 + np.random.randn(30).cumsum() * 2,
    'MSFT': 380 + np.random.randn(30).cumsum() * 3,
}, index=dates)

# Real data (sensitive)
np.random.seed(1337)
real_stocks = pd.DataFrame({
    'AAPL': 175 + np.random.randn(30).cumsum() * 3,
    'GOOGL': 165 + np.random.randn(30).cumsum() * 4,
    'MSFT': 420 + np.random.randn(30).cumsum() * 5,
}, index=dates)

print("Mock:", mock_stocks.shape)
print("Real:", real_stocks.shape)

Mock: (30, 3)
Real: (30, 3)


## Step 3: Publish Twin

In [4]:
stocks = Twin(public=mock_stocks, private=real_stocks, name="stocks")
session.remote_vars["stocks"] = stocks
print("Published 'stocks' twin!")


⚠️  Twin 'stocks' .public has unloaded data.
   Call .load() to load it, or approve now:
🌍 Loading PUBLIC data for Twin 'stocks'
   ✓ Loading data...
   ✓ Loaded: bytes
🌍 Using PUBLIC data from Twin 'stocks...'
📢 Published Twin 'stocks' (public side available at: /Users/madhavajay/dev/biovault-desktop/workspace2/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/7f01f8db8e4f/data/5340f86a596041999ffe9c5ea2beefc0.beaver)
Published 'stocks' twin!


## Step 4: Wait for Analysis Request

**Run DS notebook Steps 1-4 now!**

In [5]:
session.inbox()

InboxView(/Users/madhavajay/dev/biovault-desktop/workspace2/biovault/sandbox/client1@sandbox.local/datasites/client2@sandbox.local/shared/biovault/sessions/7f01f8db8e4f): empty

In [6]:
request = bv.wait_for_request(stocks, timeout=300)
request

⏳ Waiting for request on 'stocks'...
📬 Request received: request_analyze_stocks_for_result
   From: client2@sandbox.local


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⚡ Action: analyze_stocks_result
   Request from: client2@sandbox.local
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📋 Function: analyze_stocks

⚙️  Static Parameters:
  • arg[0]: dict = {'_beaver_twin_ref': True, 'twin_id': '3165ace1...

💡 Next Steps:
   .run_mock()   → Test on mock/public data (safe preview)
   .run()        → Execute on real/private data
   .run_both()   → Run on both mock & real for comparison
   .reject(msg)  → Decline this request

🆔 IDs: comp=54750c3b7069... result=66ae6fbe6446...
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 5: Run and Review

In [7]:
# Test on mock first
mock_result = request.run_mock()
print("Mock result:", mock_result.data.public)

🧪 Testing on mock/public data...
🔍 [post-execution] Figures to capture: 0
🔍 [post-execution] Already captured IDs: set()
Mock result: {'mean': {'AAPL': 149.43254053315178, 'GOOGL': 133.8453373365875, 'MSFT': 380.31000159139916}, 'std': {'AAPL': 6.608426074041916, 'GOOGL': 4.555747397577688, 'MSFT': 3.844975241790074}, 'returns': {'AAPL': -8.134289150778095, 'GOOGL': -4.370665831104405, 'MSFT': 0.6860564350308485}}


In [8]:
# Show captured output
mock_result.data.show_output("public")
mock_result.data.show_figures("public")

📤 Public output:
  stdout: Analyzing 30 days of stock data...
Columns: ['AAPL', 'GOOGL', 'MSFT']

Returns:
  AAPL: -8.13%
  GOOGL: -4.37%
  MSFT: 0.69%

📊 No public figures captured


In [9]:
# Run on real data and approve
result = request.run_both()
print("Real result:", result.data.private)
result.approve()

🧪 Step 1/2: Testing on mock/public data...
🧪 Testing on mock/public data...
🔍 [post-execution] Figures to capture: 0
🔍 [post-execution] Already captured IDs: set()
🔒 Step 2/2: Executing on real/private data...
✓ Private result: dict
Real result: {'mean': {'AAPL': 161.76203962932763, 'GOOGL': 172.7897777603486, 'MSFT': 400.05530438982765}, 'std': {'AAPL': 5.845337258141274, 'GOOGL': 7.50033652404769, 'MSFT': 10.8057291140962}, 'returns': {'AAPL': -13.159321692232925, 'GOOGL': -0.23861379852335488, 'MSFT': -7.100388177323206}}
✅ Approving result for: analyze_stocks_result (twin_id=66ae6fbe6446..., comp_id=54750c3b7069..., result_id=66ae6fbe6446)
   Sending to: client2@sandbox.local
🔒 Using PRIVATE data from Twin 'analyze_stocks_result...'
✓ Result sent to session folder: /Users/madhavajay/dev/biovault-desktop/workspace2/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/7f01f8db8e4f
💡 They can load it with: bv.inbox()['analyze_stocks_result'].

PosixPath('/Users/madhavajay/dev/biovault-desktop/workspace2/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/7f01f8db8e4f/cee56d42a76f41e4980276667ae7904d.beaver')

## Step 6: Live Variables Demo

In [10]:
import time

progress = Twin(public={"step": 0, "status": "starting"}, private=None, name="progress")
progress.enable_live(interval=1.0)
session.remote_vars["progress"] = progress

# Simulate work with progress updates
for i, msg in enumerate(["Loading...", "Processing...", "Analyzing...", "Done!"]):
    progress.public = {"step": i+1, "total": 4, "status": msg}
    session.remote_vars["progress"] = progress
    print(f"Step {i+1}/4: {msg}")
    time.sleep(2)

progress.disable_live()
print("\nComplete!")

🟢 Live sync enabled (read-only, every 1.0s)
🌍 Using PUBLIC data from Twin 'progress...'
📢 Published Twin 'progress' (public side available at: /Users/madhavajay/dev/biovault-desktop/workspace2/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/7f01f8db8e4f/data/10cfc86d985c4f8a8ffe616fd3509820.beaver)
🌍 Using PUBLIC data from Twin 'progress...'
📢 Published Twin 'progress' (public side available at: /Users/madhavajay/dev/biovault-desktop/workspace2/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/7f01f8db8e4f/data/d8eaa57bb1a04af7959e51b90500c8e7.beaver)
Step 1/4: Loading...
🌍 Using PUBLIC data from Twin 'progress...'
  📢 Re-published to public registry
🌍 Using PUBLIC data from Twin 'progress...'
📢 Published Twin 'progress' (public side available at: /Users/madhavajay/dev/biovault-desktop/workspace2/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/7f

## Step 7: Handle Plot Request

**Run DS notebook Step 7 now!**

In [11]:
plot_request = bv.wait_for_request(stocks, timeout=300)
plot_request

📬 Found existing request: request_analyze_stocks_for_result
   From: client2@sandbox.local


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⚡ Action: analyze_stocks_result
   Request from: client2@sandbox.local
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📋 Function: analyze_stocks

⚙️  Static Parameters:
  • arg[0]: dict = {'_beaver_twin_ref': True, 'twin_id': '3165ace1...

💡 Next Steps:
   .run_mock()   → Test on mock/public data (safe preview)
   .run()        → Execute on real/private data
   .run_both()   → Run on both mock & real for comparison
   .reject(msg)  → Decline this request

🆔 IDs: comp=54750c3b7069... result=66ae6fbe6446...
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
plot_result = plot_request.run_both()
plot_result.data.show_figures("both")
plot_result.approve()
print("Plot result approved!")

🧪 Step 1/2: Testing on mock/public data...
🧪 Testing on mock/public data...
🔍 [post-execution] Figures to capture: 0
🔍 [post-execution] Already captured IDs: set()
🔒 Step 2/2: Executing on real/private data...
✓ Private result: dict
✅ Approving result for: analyze_stocks_result (twin_id=66ae6fbe6446..., comp_id=54750c3b7069..., result_id=66ae6fbe6446)
   Sending to: client2@sandbox.local
🔒 Using PRIVATE data from Twin 'analyze_stocks_result...'
✓ Result sent to session folder: /Users/madhavajay/dev/biovault-desktop/workspace2/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/7f01f8db8e4f
💡 They can load it with: bv.inbox()['analyze_stocks_result'].load()
Plot result approved!


## Summary

- `wait_for_request(twin)` - Reliably wait for requests
- `.show_output()` / `.show_figures()` - View captured output
- `enable_live()` - Real-time variable sync